In [1]:
import pandas as pd
import numpy as np
import re
import ast
import uuid
from datetime import datetime, timedelta
from collections import Counter
from tqdm import tqdm

tqdm.pandas()

In [2]:
meta_path = '/data/log-data-2024/1.meta_rot_preprocessing_ksc/data/Meta_241127.csv'
search_log_path = '/data/log-data-2024/1.meta_rot_preprocessing_ksc/data/search_log_241127.csv'
recommend_path = '/data/log-data-2024/recommend_click_log_all_09_11.csv'
sequence_output_path = '/data/log-data-2024/2.sequence_generate_ksc/data/input_241127.txt'
sequence_device_path = '/data/log-data-2024/2.sequence_generate_ksc/data/sequence_device_match_241127.csv'
title_id_match_path = '/data/log-data-2024/2.sequence_generate_ksc//data/title_id_match_241127.csv'

In [3]:
meta = pd.read_csv(meta_path, index_col=0)
meta.reset_index(drop=True, inplace=True)
print(meta.shape)

meta = meta.rename(columns = {'contents_id': 'content_id', 'contents_set_id':'content_set_id', 'new_contents_id':'new_content_id'})

meta.head(2)

/tmp/ipykernel_39340/3083755512.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  meta = pd.read_csv(meta_path, index_col=0)


(1459901, 13)


,content_set_id,content_id,execution_info,domain,description,thumbnail_info,actor_info,director_info,cntry_code,document_id,title_name,genre_info,new_content_id
0,com.lge.meta.crawler.vudu.VuduCrawler,60945,20 vudu com.lge.meta.crawler.vudu.VuduCrawler ...,movie,You've always got to expect the unexpected and...,#vudu/60945-338#ADDSIZE#S-1X-1,"['Ethan Embry', 'Angus Scrimm', 'Heather Feene...",1309824 Don Coscarelli Director,US,c39582384bf71fc8516093ea679bd2d9f634d9e0c741a8...,Masters of Horror: Incident On and Off a Mount...,['호러'],0.0
1,com.lge.meta.crawler.vudu.VuduCrawler,60946,20 vudu com.lge.meta.crawler.vudu.VuduCrawler ...,movie,Jamie (Henry Thomas of GANGS OF NEW YORK) is a...,#vudu/60946-338#ADDSIZE#S-1X-1,"['Jake D. Smith', 'Henry Thomas', 'Lucie Lauri...",NaN,US,94fd0770850ade9cdf87c95b8bb1fe435d27a775e74af7...,Masters of Horror: Chocolate: Mick Garris,['호러'],1.0


In [4]:
app_id = pd.read_csv('/data/log-data-2024/appId_setId_mapping.csv')
app_id.head(2)

,app_id,contents_set_id
0,amazon,com.lge.meta.crawler.amazon.AmazonCrawler_US
1,hulu,com.lge.meta.crawler.hulu.CrawlegoHuluCrawler


In [ ]:
recommend_click = pd.read_csv(recommend_path)
recommend_click = pd.merge(recommend_click, app_id, left_on = 'message_value_1', right_on='app_id', how='inner')

recommend_click = recommend_click[['device_id', 'log_time', 'message_value_1', 'message_value_2', 'message_value_4', 'contents_set_id']]

recommend_click = recommend_click.rename(columns = {'message_value_1': 'app_id', 'message_value_2': 'content_id', 'contents_set_id':'content_set_id'})

recommend_click = pd.merge(recommend_click, meta[['content_id', 'content_set_id', 'new_content_id']], on=['content_id', 'content_set_id'], how='left')


print(recommend_click.shape)
recommend_click.tail(2)

(562849, 7)


,device_id,log_time,app_id,content_id,message_value_4,content_set_id,new_content_id
562847,o9BGxmUAx9ZiIpUl9Ss3NnY40Fo46bKNOAUBnU/ch4E=,2024-09-03 14:19:15.000,com.curiositystream.app,series-1177,Thomas & Friends,com.lge.meta.crawler.curiositystream.Curiosity...,NaN
562848,JR4throRyr2j3hFT6JsQHw5AergZ7ucRdDqKCftrxUc=,2024-09-01 21:01:49.000,com.curiositystream.app,series-174,After Braveheart,com.lge.meta.crawler.curiositystream.Curiosity...,NaN


In [ ]:
recommend_click[recommend_click['new_content_id'].isnull()] 

,device_id,log_time,app_id,content_id,message_value_4,content_set_id,new_content_id
1,O2K3IXzMYzfbYRQvSemtoqsmW+fwG2So9pu5QaN9H/k=,2024-10-03 00:52:07.000,amazon,amzn1.dv.gti.b5f4c96d-4773-452d-8b55-180772a0f5c9,From,com.lge.meta.crawler.amazon.AmazonCrawler_US,NaN
4,xSPGAC59GSLWbfpncCUAMy0XyJY90MFUVl9K6P6/R50=,2024-09-18 00:46:25.000,amazon,amzn1.dv.gti.1d3e1499-2cef-438d-959b-ca240a939d65,The Old Man,com.lge.meta.crawler.amazon.AmazonCrawler_US,NaN
6,SNj8RAJJkChw5bD6eJve9RIToo8iVOpbK1GMm1J8Lxk=,2024-09-13 00:59:16.000,amazon,amzn1.dv.gti.91e49ebe-21b2-48ec-9b92-0b50e57d2392,Sonic the Hedgehog 2,com.lge.meta.crawler.amazon.AmazonCrawler_US,NaN
11,HUzLkC4Jx3OU77VaHppb09YDryKpEJmlSGZhIVCGxOg=,2024-10-03 00:01:31.000,amazon,amzn1.dv.gti.704dcd96-2364-4ddd-b10e-f3dac081aef9,Matlock,com.lge.meta.crawler.amazon.AmazonCrawler_US,NaN
17,DXxVmI+8kp/DwmFoK/KPGiIrKMiOj+eQcz5BwlYkmQE=,2024-10-03 01:04:46.000,amazon,amzn1.dv.gti.a528b73d-7e40-4e1b-9721-2ae903f79dd3,9-1-1,com.lge.meta.crawler.amazon.AmazonCrawler_US,NaN
...,...,...,...,...,...,...,...
562844,VO9ShxIZzoCNsmVZLfQO3BqbaEqsklI1cr+CXHKJNiA=,2024-10-04 22:30:10.000,com.discovery.tlc,go-video-2149709,Sweet Home Sextuplets,com.lge.meta.crawler.discoverypluse.DiscoveryP...,NaN
562845,VO9ShxIZzoCNsmVZLfQO3BqbaEqsklI1cr+CXHKJNiA=,2024-10-04 22:29:01.000,com.discovery.tlc,go-video-5344163,Breaking Amish,com.lge.meta.crawler.discoverypluse.DiscoveryP...,NaN
562846,liSNrv5OQpPadLFdzBQ4EFLfShmiBzc/ZtkGbsmiPuU=,2024-10-10 20:48:32.000,com.discovery.tlc,go-video-5391049,1000-lb Sisters,com.lge.meta.crawler.discoverypluse.DiscoveryP...,NaN
562847,o9BGxmUAx9ZiIpUl9Ss3NnY40Fo46bKNOAUBnU/ch4E=,2024-09-03 14:19:15.000,com.curiositystream.app,series-1177,Thomas & Friends,com.lge.meta.crawler.curiositystream.Curiosity...,NaN


In [15]:
meta[meta['content_id'] == 'amzn1.dv.gti.704dcd96-2364-4ddd-b10e-f3dac081aef9']

,content_set_id,content_id,execution_info,domain,description,thumbnail_info,actor_info,director_info,cntry_code,document_id,title_name,genre_info,new_content_id


In [23]:
meta[meta['title_name']== 'Sonic the Hedgehog 2'][['content_set_id', 'content_id', 'execution_info', 'title_name']]



,content_set_id,content_id,execution_info,title_name
1154447,com.lge.meta.crawler.appletv.AppleTVCrawler_US,https://tv.apple.com/us/movie/sonic-le-herisso...,50 com.apple.appletv com.lge.meta.crawler.appl...,Sonic the Hedgehog 2
1247087,com.lge.meta.crawler.tms.TmsCrawler,TMS18644345,20 amazon_html com.lge.meta.crawler.amazon.Ama...,Sonic the Hedgehog 2
1258192,com.lge.meta.crawler.amazon.AmazonCrawler_US,amzn1.dv.gti.09ce3e5c-97c6-48f8-993e-2b2aedf15522,20 amazon_html com.lge.meta.crawler.amazon.Ama...,Sonic the Hedgehog 2


In [26]:
meta[meta['title_name']== 'Sonic the Hedgehog 2'].iloc[1].execution_info

'20\x0bamazon_html\x0bcom.lge.meta.crawler.amazon.AmazonCrawler_US\x0bamzn1.dv.gti.09ce3e5c-97c6-48f8-993e-2b2aedf15522\x0c50\x0bamazon\x0bcom.lge.meta.crawler.amazon.AmazonCrawler_US\x0bamzn1.dv.gti.09ce3e5c-97c6-48f8-993e-2b2aedf15522\x0c50\x0bamazon-pjtr\x0bcom.lge.meta.crawler.amazon.AmazonCrawler_US\x0bamzn1.dv.gti.09ce3e5c-97c6-48f8-993e-2b2aedf15522\x0c50\x0bamazon.html\x0bcom.lge.meta.crawler.amazon.AmazonCrawler_US\x0bamzn1.dv.gti.09ce3e5c-97c6-48f8-993e-2b2aedf15522\x0c20\x0bvudu\x0bcom.lge.meta.crawler.vudu.VuduCrawler\x0b2010680\x0c50\x0bvudu\x0bcom.lge.meta.crawler.vudu.VuduCrawler\x0b2010680'

In [7]:
recommend_click = recommend_click[recommend_click['new_content_id'].notnull()]

In [8]:
# recommend_click = recommend_click[(recommend_click['log_time'] >= '2024-08-01') & (recommend_click['log_time'] < '2024-09-01')]

In [9]:
search = pd.read_csv(search_log_path)
search = pd.merge(search, meta[['content_id', 'content_set_id', 'new_content_id']], on=['content_id', 'content_set_id'], how='left')
print(search.shape)
search.tail(2)

(1638672, 6)


,device_id,log_time,content_id,content_set_id,api_logging,new_content_id
1638670,WFpqNihBiCXqYgICSw//gwsJ7H1RG/9llpsjfgPZRHs=,2024-10-31 21:59:34.253,TMS17290385,com.lge.meta.crawler.tms.TmsCrawler,logging,587547.0
1638671,wzEqRYvyesZFKHVdzsWpViezk9NUfSaFSerK5TAdYLU=,2024-10-31 21:00:00.294,TMS18975942,com.lge.meta.crawler.tms.TmsCrawler,logging,627528.0


In [16]:
def data_preprocessing(recommend_click):
    # 정렬
    recommend_click = recommend_click.sort_values(by=['device_id', 'log_time'])
    
    # 이상치 제거
    recommend_click['log_time'] = recommend_click['log_time'].progress_apply(lambda x: x.split('.')[0])
    recommend_click['log_time'] = pd.to_datetime(recommend_click['log_time'], format='%Y-%m-%d %H:%M:%S')
    
    # log_time을 datetime으로 변환
    recommend_click['log_time'] = pd.to_datetime(recommend_click['log_time'])

    return recommend_click



def data_preprocessing_search(search):
    # 정렬
    search = search.sort_values(by=['device_id', 'log_time'])
    # 이상치 제거
    search['log_time'] = search['log_time'].progress_apply(lambda x: x.split('.')[0])
    search['log_time'] = pd.to_datetime(search['log_time'], format='%Y-%m-%d %H:%M:%S')

    # search_df = search[['device_id', 'message_value_4', 'log_time']].copy()
    
    # log_time을 datetime으로 변환
    search['log_time'] = pd.to_datetime(search['log_time'])
    
    return search



def generate_sequences(df, col_name, day_term=30):
    time_term = timedelta(days=day_term)
    # df['log_time'] = pd.to_datetime(df['log_time'])
    df = df.sort_values(by='log_time')
    sequences = []
    current_sequence = []
    previous_time = None
    
    for _, row in df.iterrows():
        if previous_time is None or (row['log_time'] - previous_time) >= time_term:
            if len(current_sequence) > 2:
                sequences.append(current_sequence)
            current_sequence = [row[col_name]]
        else:
            current_sequence.append(row[col_name])
        previous_time = row['log_time']
    
    if len(current_sequence) > 2: 
        sequences.append(current_sequence)
    
    return sequences


def convert_list_to_txt(click_df, col_name, day_term=30):
    # 각 numeric_device_id에 대해 시퀀스를 생성하고 리스트로 변환
    grouped = click_df.groupby('device_id')
    
    total_sequences = []
    sequence_device = {}  # device_id와 시퀀스 인덱스 매핑을 위한 딕셔너리
    idx = 0  # 시퀀스 인덱스 초기화

    for device_id, group in grouped:
        # device_id에 대한 시퀀스를 생성
        sequences = generate_sequences(group, col_name, day_term)
        total_sequences.extend(sequences)
        
        # 각 시퀀스에 대해 시퀀스 번호와 device_id 매핑 저장
        for _ in sequences:
            sequence_device[idx] = device_id
            idx += 1
    
    # 결과를 문자열로 변환하여 텍스트 파일에 저장
    result = []
    for idx, item in enumerate(total_sequences):
        content_string = str(idx) + ' ' + ' '.join(map(str, item)) + '\n'
        result.append(content_string)

    # 시퀀스와 device_id 매핑을 DataFrame으로 변환하고 CSV로 저장
    sequence_device_df = pd.DataFrame(list(sequence_device.items()), columns=['seq_idx', 'device_id'])
    sequence_device_df.to_csv(sequence_device_path, index=False)
    
    # 시퀀스 데이터를 텍스트 파일로 저장
    with open(sequence_output_path, 'w') as file:
        file.writelines(result)




def main(recommend_click, search, day_term=30):
    print('--------recommend_click 전처리--------')
    rec_temp = data_preprocessing(recommend_click)

    print('-------------search 전처리-------------')
    search_temp = data_preprocessing_search(search)

    temp = pd.concat([rec_temp, search_temp], axis=0)

    print('---recommend_click, search 결합 완료---')

    temp['log_time'] = pd.to_datetime(temp['log_time'], format='%Y-%m-%d %H:%M:%S.%f')
    result = temp[temp['new_content_id'].notna()]
    
    print('-----------list 파일로 변환------------')
    convert_list_to_txt(result, 'new_content_id', day_term)

    return result

In [17]:
result = main(recommend_click, search, 30)
result.head(3)

--------recommend_click 전처리--------


100%|██████████| 234128/234128 [00:00<00:00, 753585.47it/s]


-------------search 전처리-------------


100%|██████████| 1638672/1638672 [00:02<00:00, 760332.31it/s]


---recommend_click, search 결합 완료---
-----------list 파일로 변환------------


,device_id,log_time,app_id,content_id,message_value_4,content_set_id,new_content_id,api_logging
511851,+++W5OAUCVG7OcDazfYT1Z8OvZ0OwQLD189Y1NxABQ0=,2024-09-01 17:51:28,com.tubitv.ott.tubi,content_id=642249&utm_source=search&utm_medium...,The Graduate,com.lge.meta.crawler.tubi.TubiCrawler_US,88542.0,NaN
559343,++/Nl/IaiaJnbpQNMKld6z0GhsVE8otn7M1o+pl53h0=,2024-10-29 17:11:39,vudu,926204,Terrifier,com.lge.meta.crawler.vudu.VuduCrawler,666519.0,NaN
136733,++0cYSHrLKkEkdqYy9EK0rrJSk9byDe/sd9MftYaCqo=,2024-09-21 22:34:47,com.apple.appletv,https://tv.apple.com/us/movie/lightyear/umc.cm...,Lightyear,com.lge.meta.crawler.appletv.AppleTVCrawler_US,528519.0,NaN


In [23]:
unique_result = result[['message_value_4', 'contents_id']]
unique_result.to_csv(title_id_match_path, index=False)

### sequence 확인

In [20]:
import pandas as pd

def view_txt(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    data = []
    for line in lines:
        parts = line.strip().split()
        sequence_id = int(parts[0])
        sequence = list(map(lambda x: int(float(x)), parts[1:]))
        data.append([sequence_id, sequence])
    
    df = pd.DataFrame(data, columns=['sequence_num', 'sequence'])
    return df

def avg_seq_len(df):
    sequence_lengths = df['sequence'].apply(len)
    average_length = sequence_lengths.mean()
    return average_length

In [21]:
input = view_txt(sequence_output_path)
print(input.shape)
input.head()

(73505, 2)


,sequence_num,sequence
0,0,"[588756, 151926, 588756]"
1,1,"[596461, 596461, 596461, 629694, 627713]"
2,2,"[595073, 595073, 595071, 612889]"
3,3,"[567397, 184872, 598367, 564650, 564650, 58700..."
4,4,"[285772, 627528, 63079]"


In [ ]:
result.contents_id = result.contents_id.astype(int)

In [ ]:
# contents_id의 값을 문자열과 정수 형태로 변환
result_values = set(result['contents_id'].astype(str).values)

In [33]:
# 데이터 확인 : True면 다 있는 거
all(all(str(item) in result_values for item in row['sequence']) 
    for _, row in input.iterrows())

True

In [34]:
input

,sequence_num,sequence
0,0,"[20029962, 3625299, 3625299, 8000398, 3625299,..."
1,1,"[16493930, 8174, 13472534]"
2,2,"[184133, 10643, 24413]"
3,3,"[186698, 186698, 186698]"
4,4,"[8000398, 9286627, 13285, 14158914]"
...,...,...
62043,62043,"[20029962, 20029962, 9968629]"
62044,62044,"[12343481, 12343481, 12343481]"
62045,62045,"[184061, 185445, 10492774]"
62046,62046,"[18863434, 12906646, 12906646, 12906646, 18863..."


In [ ]:
result.contents_id.nunique()

21723